**NARGEOT** Melvine
**PATTE** Mathys

# Projet Outil Informatique

In [ ]:
%pip install requests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests

## Données

### Sources

https://www.ngdc.noaa.gov/hazel/hazard-service/api/v1/volcanoes?page=1

https://volcano.si.edu/search_eruption.cfm

https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions/

https://data.cerdi.uca.fr/erup-vol/

https://public.opendatasoft.com/explore/dataset/significant-volcanic-eruption-database/export/

https://www.ngdc.noaa.gov/hazel/view/hazards/volcano/event-search/

https://berkeleyearth.org/data/

### Import

In [ ]:
url = ("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/significant-volcanic-eruption-database/exports/json?lang=fr&timezone=Europe%2FBerlin")

response = requests.get(url)

with open("datasets/volcanic.json", mode="wb") as file:
    file.write(response.content)

In [ ]:
volcanic = pd.read_json("datasets/volcanic.json")

### Cleaning

## Analyse

In [ ]:
monthly_t = pd.read_json("datasets/monthly_t.json") # Reading the json file of global monthly temperature anomaly

monthly_t = monthly_t.loc[monthly_t["Source"] == "GISTEMP"] # Only keep values from GISTEMP (nasa)
monthly_t.reset_index(inplace=True) # Reset row index to manipulate rows easily

# Splitting the date in format yyyy-mm-dd into Year and Month to use it with other data later
year = [monthly_t["Date"][date].year for date in range(len(monthly_t["Date"]))] # Create a 'Year' column from the timestamp given in 'Date'
monthly_t["Year"] = year

month = [monthly_t["Date"][date].month for date in range(len(monthly_t["Date"]))] # Create a 'Month' column from the timestamp given in 'Date'
monthly_t["Month"] = month

monthly_t = monthly_t.drop(['Date', 'Source'], axis=1) # Delete useless rows for later use
monthly_t = monthly_t.reindex(columns=['Year', 'Month', 'Mean']) # Put date first then temperature anomaly
monthly_t.sort_values(by=['Year','Month'], inplace=True) # Sort by ascending date
monthly_t.reset_index(inplace=True, drop=True) # Reset index for readability 
monthly_t
